In [1]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
# import config
import re
from sqlalchemy import create_engine
import unicodedata

In [2]:
# Import json and convert to DF
url = 'https://www.thecocktaildb.com/api/json/v2/9973533/search.php?s'
response = requests.get(url).json()
drinks_df = pd.json_normalize(response['drinks'], max_level=1)
drinks_df.head()

,idDrink,strDrink,strDrinkAlternate,strTags,strVideo,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,...,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strImageSource,strImageAttribution,strCreativeCommonsConfirmed,dateModified
0,15997,GG,None,None,None,Ordinary Drink,None,Optional alcohol,Collins Glass,Pour the Galliano liqueur over ice. Fill the r...,...,None,None,None,None,None,None,None,None,No,2016-07-18 22:06:00
1,17222,A1,None,None,None,Cocktail,None,Alcoholic,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",...,None,None,None,None,None,None,None,None,No,2017-09-07 21:42:09
2,13501,ABC,None,None,None,Shot,None,Alcoholic,Shot glass,Layered in a shot glass.,...,None,None,None,None,None,None,None,None,No,2016-08-31 19:32:08
3,17203,Kir,None,"IBA,ContemporaryClassic",None,Ordinary Drink,None,Alcoholic,Wine Glass,Add the crème de cassis to the bottom of the g...,...,None,None,None,None,None,None,None,None,No,2017-09-02 17:38:14
4,14229,747,None,None,None,Shot,None,Alcoholic,Shot glass,"pour kaluha, then Baileys, then Frangelico not...",...,None,None,None,None,None,None,None,None,No,2016-08-31 19:28:26


In [3]:
# get only the columns we need
measure_df = drinks_df[['idDrink', 'strDrink',  'strIngredient1', 'strMeasure1', 'strIngredient2', 'strMeasure2',
                        'strIngredient3', 'strMeasure3', 'strIngredient4', 'strMeasure4','strIngredient5', 'strMeasure5',
                        'strIngredient6', 'strMeasure6', 'strIngredient7', 'strMeasure7','strIngredient8', 'strMeasure8',
                        'strIngredient9', 'strMeasure9', 'strIngredient10', 'strMeasure10', 'strIngredient11', 'strMeasure11',
                        'strIngredient12', 'strMeasure12']]

measure_df

,idDrink,strDrink,strIngredient1,strMeasure1,strIngredient2,strMeasure2,strIngredient3,strMeasure3,strIngredient4,strMeasure4,...,strIngredient8,strMeasure8,strIngredient9,strMeasure9,strIngredient10,strMeasure10,strIngredient11,strMeasure11,strIngredient12,strMeasure12
0,15997,GG,Galliano,2 1/2 shots,Ginger ale,None,Ice,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,17222,A1,Gin,1 3/4 shot,Grand Marnier,1 Shot,Lemon Juice,1/4 Shot,Grenadine,1/8 Shot,...,None,None,None,None,None,None,None,None,None,None
2,13501,ABC,Amaretto,1/3,Baileys irish cream,1/3,Cognac,1/3,None,None,...,None,None,None,None,None,None,None,None,None,None
3,17203,Kir,Creme de Cassis,1 part,Champagne,5 parts,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,14229,747,Kahlua,1/3 part,Baileys irish cream,1/3 part,Frangelico,1/3 part,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,16100,Amaretto Stone Sour Alternative,Sour mix,2 oz,Amaretto,2 oz,Tequila,2 oz,Orange juice,Add splash,...,None,None,None,None,None,None,None,None,None,None
614,12658,Banana Strawberry Shake Daiquiri,Strawberries,1/2 lb frozen,Banana,1 frozen,Apple juice,2 cups fresh,None,None,...,None,None,None,None,None,None,None,None,None,None
615,16984,Radioactive Long Island Iced Tea,Rum,1 oz,Vodka,1 oz,Tequila,1 oz,Gin,1 oz,...,Malibu rum,1 oz,None,None,None,None,None,None,None,None
616,14029,57 Chevy with a White License Plate,Creme de Cacao,1 oz white,Vodka,1 oz,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [4]:
### Create table for converting all units to ounces
measurement_conversions = {'Units': ['','cL','qt','shot','spoons','Dash','Cup','cans','quart','drop','L','Dashes','jigger','bottle',
                'fifth','top','part','Fill','shots','beaten','tablespoons','inch','gr','tbsp','handful','pinches',
                'kg','tsp','cubes','cup','tblsp','splash','dl','can','pint','parts','full','Shot','Full','Bacardi',
                'jiggers','drops','Pinch','cube','oz','splashes','cl','Float','lb','ml','bottles','scoops','Squeeze',
                'fill','package','packages','dash','dashes','gal','seltzer','Juice','cups','Top'],
                             'multiplier': [0,0.338,32,1.5,0.5,0.25,8,12,32,0.01,33.8,0.25,1.5,12,
                                           25.5,1,1.5,1,1.5,2,0.5,1,0.035,0.5,1,0.15,
                                           35.274,0.167,0.5,8,0.5,0.5,3.38,12,16,1.5,1,1.5,1,1.5,
                                           1.5,0.01,0.15,0.5,1,0.5,0.338,1,16,0.0338,12,2,0.25,
                                           1,1,1,0.25,0.25,124,1,1,8,1]}
measurement_conversions_df = pd.DataFrame.from_dict(measurement_conversions)

measurement_conversions_df.head(10)

,Units,multiplier
0,,0.000
1,cL,0.338
2,qt,32.000
3,shot,1.500
4,spoons,0.500
5,Dash,0.250
6,Cup,8.000
7,cans,12.000
8,quart,32.000
9,drop,0.010


In [5]:
### Converting measurments amounts to ounces - use this in apply function inside the "split" apply function
def units_to_oz(row):
    unit = row["Units"]
    #if unit in measurement_conversions_df, then
    if len(measurement_conversions_df.loc[measurement_conversions_df["Units"] == unit]) > 0:
        mult = measurement_conversions_df.loc[measurement_conversions_df["Units"] == unit, "multiplier"].item()
        t = row["Total"]
        row["Total"] = t * mult
        if row["Total"] == None:
            row["Total"] = 0
    else:
        row["Total"] = 0
    return row

In [6]:
### Apply function to split up strMeasure#s and convert to oz
# First separate numbers from words
count = 0
def split(df_column):
    global count
    count += 1
    measurements = []
    units = []   
    try: 
        for index, string_list in df_column.items():
            if (string_list):
                    wordsRegex = "[a-zA-z]+"
                    numbersRegex = "(\d [\d\/\d]*)"
                    words = re.search(wordsRegex, string_list)
                    measure = re.search(numbersRegex, string_list)
                    if (words):
                        units.append(words.group(0))
                    else:
                        units.append("")
                    if (measure):
                        measurements.append(measure.group(0))
                    else:
                        measurements.append("")
            else:
                measurements.append("")
                units.append("")

        measurements_df = pd.DataFrame(
            {"Measurements": measurements, "Units": units}, 
        )

        # Next, convert the fractions to floats

        # split measurements column on the space, rename columns 
        split_measurements_df = pd.DataFrame(measurements_df['Measurements'].str.split(' ',1).tolist(),
                                         columns = ['Measurement_a','Measurement_b'])

        # Potential fractions to be dealt with: ['1/2', '3/4', '2/3', '1/4', '1/3']
        # replace with decimals
        split_measurements_df['Measurement_b'] = split_measurements_df['Measurement_b'].replace(['1/2','3/4','2/3','1/4','1/3'],
                                                                                                [0.5,0.75,0.66,0.25,0.33])
        # convert string to number
        split_measurements_df['Measurement_a'] = pd.to_numeric(split_measurements_df['Measurement_a'], errors='coerce')
        split_measurements_df['Measurement_b'] = pd.to_numeric(split_measurements_df['Measurement_b'], errors='coerce')

        # make all NANs = 0
        split_measurements_df['Measurement_b'] = split_measurements_df['Measurement_b'].fillna(0)

        # create a new column with total
        split_measurements_df['Total'] = split_measurements_df['Measurement_a'] + split_measurements_df['Measurement_b']

        # add back the units column
        split_measurements_df['Units'] = units

        # Now convert to oz using apply function
        normalized_measurements_df = split_measurements_df.apply(units_to_oz, axis = 1)

        df_column = normalized_measurements_df["Total"]
    
    except:
        pass
    
    return df_column # this returns a series

In [7]:
### Use apply function on strMeasure#s columns
just_measurements = measure_df[['strMeasure1', 'strMeasure2',
       'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6',
       'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10',
       'strMeasure11', 'strMeasure12']].apply(split)
clean_measurements = just_measurements.fillna(0)
clean_measurements

,strMeasure1,strMeasure2,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12
0,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.625,1.5,6.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.500,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.500,4.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
613,2.000,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
614,32.000,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
615,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
616,1.000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# get non-meansurement columns
clean_ingredients = measure_df[['idDrink', 'strDrink', 'strIngredient1', 'strIngredient2',
                        'strIngredient3', 'strIngredient4','strIngredient5',
                        'strIngredient6','strIngredient7','strIngredient8',
                        'strIngredient9','strIngredient10','strIngredient11',
                        'strIngredient12']]
# combine ingredients with measurements
result = pd.concat([clean_ingredients, clean_measurements], axis=1)
result

,idDrink,strDrink,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,strIngredient8,...,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12
0,15997,GG,Galliano,Ginger ale,Ice,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17222,A1,Gin,Grand Marnier,Lemon Juice,Grenadine,None,None,None,None,...,6.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13501,ABC,Amaretto,Baileys irish cream,Cognac,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17203,Kir,Creme de Cassis,Champagne,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14229,747,Kahlua,Baileys irish cream,Frangelico,None,None,None,None,None,...,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,16100,Amaretto Stone Sour Alternative,Sour mix,Amaretto,Tequila,Orange juice,None,None,None,None,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
614,12658,Banana Strawberry Shake Daiquiri,Strawberries,Banana,Apple juice,None,None,None,None,None,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
615,16984,Radioactive Long Island Iced Tea,Rum,Vodka,Tequila,Gin,Triple sec,Chambord raspberry liqueur,Midori melon liqueur,Malibu rum,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
616,14029,57 Chevy with a White License Plate,Creme de Cacao,Vodka,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
### ABV TABLE
# URL of page to be scraped
abv_url = 'http://www.alcoholcontents.com/liquor/'
abv_table = pd.read_html(abv_url)
abv_df = abv_table[0]
abv_df_clean = abv_df.drop(['Cal'], axis=1)
#Rename columns
renamed_df = abv_df_clean.rename(columns={"Liquor / Cocktail / Liqueur":"Spirit", "% Alc (ABV)":"ABV"})
renamed_df.head()

,Spirit,ABV
0,Advocaat,17
1,Aftershock,30
2,Aftershock Cinnamon Schnapps,40
3,Aftershock Cool Citrus Mint Schnapps,40
4,Aguardiente,30


In [10]:
# adding some common spirits that were not in the scraped table
common_spirits = [{"Spirit": "Baijiu", "ABV": 47},
                {"Spirit": "Bourbon", "ABV": 65},
                {"Spirit": "Brandy", "ABV": 42},
                {"Spirit": "Champagne", "ABV": 12},
                {"Spirit": "Cognac", "ABV": 40},                  
                {"Spirit": "Everclear", "ABV": 85},
                {"Spirit": "Gin", "ABV": 37},
                {"Spirit": "Grand Marnier", "ABV": 40},
                {"Spirit": "Grappa", "ABV": 47},
                {"Spirit": "Mezcal", "ABV": 55},
                {"Spirit": "Peach Schnapps", "ABV": 23},
                {"Spirit": "Rum", "ABV": 47},
                {"Spirit": "Sake", "ABV": 15},
                {"Spirit": "Tequila", "ABV": 45},
                {"Spirit": "Vermouth", "ABV": 19},
                {"Spirit": "Vodka", "ABV": 42},
                {"Spirit": "Whiskey", "ABV": 46},
                {"Spirit": "German Schnapps", "ABV": 30}]

spirits_df = pd.DataFrame(common_spirits)

In [11]:
# Merge spirits_df and abv_df
merge_df = pd.merge(renamed_df, spirits_df, on=["Spirit", "ABV"], how="outer")

In [12]:
abv_sort_df = merge_df.sort_values("Spirit")
abv_sort_df.head(20)

,Spirit,ABV
0,Advocaat,17
1,Aftershock,30
2,Aftershock Cinnamon Schnapps,40
3,Aftershock Cool Citrus Mint Schnapps,40
4,Aguardiente,30
5,Amadeus,5
6,Amaretto,28
7,Amaretto Di Saronno,28
8,Amaro,17
9,Amarula,17


In [13]:
### removing accent marks from spirits in abv table
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text)

In [14]:
abv_sort_df["Spirit"] = abv_sort_df["Spirit"].str.title().apply(strip_accents)
abv_sort_df.head()

,Spirit,ABV
0,Advocaat,17
1,Aftershock,30
2,Aftershock Cinnamon Schnapps,40
3,Aftershock Cool Citrus Mint Schnapps,40
4,Aguardiente,30


In [31]:
abv_sort_df.to_sql(name='abv_table', con=engine, if_exists='replace', index=False)

In [15]:
## looking for missing spirits
# abv_sort_df[abv_sort_df["Spirit"] == "Blue Curacao"]

In [16]:
# Create a blank column for drink abv calculation
result["Drink_ABV"] = 0
result

,idDrink,strDrink,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,strIngredient8,...,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12,Drink_ABV
0,15997,GG,Galliano,Ginger ale,Ice,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,17222,A1,Gin,Grand Marnier,Lemon Juice,Grenadine,None,None,None,None,...,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,13501,ABC,Amaretto,Baileys irish cream,Cognac,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,17203,Kir,Creme de Cassis,Champagne,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,14229,747,Kahlua,Baileys irish cream,Frangelico,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,16100,Amaretto Stone Sour Alternative,Sour mix,Amaretto,Tequila,Orange juice,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
614,12658,Banana Strawberry Shake Daiquiri,Strawberries,Banana,Apple juice,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
615,16984,Radioactive Long Island Iced Tea,Rum,Vodka,Tequila,Gin,Triple sec,Chambord raspberry liqueur,Midori melon liqueur,Malibu rum,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
616,14029,57 Chevy with a White License Plate,Creme de Cacao,Vodka,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [17]:
### ABV CALC
for drink in result.iterrows():
    # index of drink is located at drink[0]
    # row of drink data is located at drink[1]
    total_abv = 0
    total_measure = 0
    for ing in range(2,13):
        for spirit in abv_sort_df["Spirit"]:
            # update ingredients using .title()
            if drink[1][ing] and spirit in drink[1][ing].title():
                abv = abv_sort_df.loc[abv_sort_df["Spirit"] == spirit]["ABV"]
                abv = abv.values[0]
                amount =  drink[1][ing + 12]
                total_abv += abv * amount

    final_calculation = 0
    for meas in range(14,26):
        if drink[1][meas]:
            total_measure += drink[1][meas]
            final_calculation = total_abv/total_measure
            
    # before for loop add column to result df defaulting to 0 for final ABV
    result.at[drink[0],"Drink_ABV"] = final_calculation

In [18]:
result

,idDrink,strDrink,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,strIngredient8,...,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12,Drink_ABV
0,15997,GG,Galliano,Ginger ale,Ice,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35
1,17222,A1,Gin,Grand Marnier,Lemon Juice,Grenadine,None,None,None,None,...,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
2,13501,ABC,Amaretto,Baileys irish cream,Cognac,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,17203,Kir,Creme de Cassis,Champagne,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
4,14229,747,Kahlua,Baileys irish cream,Frangelico,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,16100,Amaretto Stone Sour Alternative,Sour mix,Amaretto,Tequila,Orange juice,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24
614,12658,Banana Strawberry Shake Daiquiri,Strawberries,Banana,Apple juice,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
615,16984,Radioactive Long Island Iced Tea,Rum,Vodka,Tequila,Gin,Triple sec,Chambord raspberry liqueur,Midori melon liqueur,Malibu rum,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,41
616,14029,57 Chevy with a White License Plate,Creme de Cacao,Vodka,None,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33


In [19]:
# Find where the code is not working
zero_abvs = result.loc[result["Drink_ABV"] == 0]
zero_abvs[['strDrink', 'strIngredient1', 'strMeasure1', 'strIngredient2', 'strMeasure2', 'Drink_ABV']]

,strDrink,strIngredient1,strMeasure1,strIngredient2,strMeasure2,Drink_ABV
2,ABC,Amaretto,0.0,Baileys irish cream,0.0,0
11,B-52,Baileys irish cream,0.0,Grand Marnier,0.0,0
15,A. J.,Applejack,1.5,Grapefruit juice,1.0,0
17,Karsk,Coffee,1.5,Grain alcohol,3.0,0
18,Melya,Espresso,0.0,Honey,0.0,0
...,...,...,...,...,...,...
602,Banana Cantaloupe Smoothie,Cantaloupe,2.0,Banana,0.0,0
607,Lassi - A South Indian Drink,Yoghurt,16.0,Water,10.0,0
609,Orange Scented Hot Chocolate,Milk,16.0,Chocolate,4.0,0
612,Grape lemon pineapple Smoothie,Grapes,8.0,Lemon,0.0,0


In [20]:
ABC = result.loc[result["strDrink"] == "ABC"]
ABC[['strDrink', 'strIngredient1', 'strMeasure1', 'strIngredient2', 'strMeasure2', 'strIngredient3', 'strMeasure3', 'Drink_ABV']]

,strDrink,strIngredient1,strMeasure1,strIngredient2,strMeasure2,strIngredient3,strMeasure3,Drink_ABV
2,ABC,Amaretto,0.0,Baileys irish cream,0.0,Cognac,0.0,0


In [21]:
username = 'postgres'
password = 'password'
rds_connection_string = f"{username}:{password}@localhost:5432/Project-2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [22]:
result.to_sql(name='cocktail_data', con=engine, if_exists='replace', index=False)

In [23]:
# result.to_csv("cocktail_data.csv", index = False, header = True)
# result.to_excel("cocktail_data_with_img.xlsx")

In [24]:
spirit_list = abv_sort_df['Spirit'].tolist()
pprint(spirit_list)

['Advocaat',
 'Aftershock',
 'Aftershock Cinnamon Schnapps',
 'Aftershock Cool Citrus Mint Schnapps',
 'Aguardiente',
 'Amadeus',
 'Amaretto',
 'Amaretto Di Saronno',
 'Amaro',
 'Amarula',
 'American Schnapps',
 'Anis Del Toro',
 'Anisette',
 'Arak',
 'Ashanti Gold',
 'Aurum',
 'Baijiu',
 'Baileys Irish Cream',
 'Becherovka',
 'Beirao',
 'Benedictine',
 'Blue Curacao',
 'Bourbon',
 'Brandy',
 'Barenjager',
 'Calisay',
 'Canton',
 'Chambord',
 'Champagne',
 'Chartreuse',
 'Cherry Heering',
 'Cloudberry Liqueur',
 'Coconut Rum',
 'Cognac',
 'Cointreau',
 'Creme De Banane',
 'Creme De Cacao',
 'Creme De Cassis',
 'Creme De Cerise',
 'Creme De Menthe',
 'Creme De Myrtille',
 'Creme De Noyaux',
 'Creme De Rose',
 'Creme De Violette',
 "Creme De Y'Vette",
 'Curacao',
 'Cynar',
 'Damiana',
 'Destinee',
 'Drambuie',
 'Drumgray Highland Cream Liqueur',
 'Dry Orange Curacao',
 'Durango',
 'Everclear',
 'Framboise',
 'Frangelico',
 'Frigola',
 'Galliano',
 'German Schnapps',
 'Gin',
 'Godiva Choc

In [25]:
drinks_df = result

In [26]:
# strIngredient1
spirits_ing1 = drinks_df[drinks_df["strIngredient1"].isin(spirit_list)]
ing_cnt_df_1=spirits_ing1.groupby('strIngredient1').nunique()
ing1_cnt = ing_cnt_df_1[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient1":"spirit","idDrink":"count1"}).set_index("spirit", drop=False)
ing1_cnt.head()

# strIngredient2
spirits_ing2 = drinks_df[drinks_df["strIngredient2"].isin(spirit_list)]
ing_cnt_df_2=spirits_ing2.groupby('strIngredient2').nunique()
ing2_cnt = ing_cnt_df_2[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient2":"spirit","idDrink":"count2"}).set_index("spirit", drop=False)
ing2_cnt.head()

# strIngredient3
spirits_ing3 = drinks_df[drinks_df["strIngredient3"].isin(spirit_list)]
ing_cnt_df_3=spirits_ing3.groupby('strIngredient3').nunique()
ing3_cnt = ing_cnt_df_3[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient3":"spirit","idDrink":"count3"}).set_index("spirit", drop=False)
ing3_cnt.head()

# strIngredient4
spirits_ing4 = drinks_df[drinks_df["strIngredient4"].isin(spirit_list)]
ing_cnt_df_4=spirits_ing4.groupby('strIngredient4').nunique()
ing4_cnt = ing_cnt_df_4[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient4":"spirit","idDrink":"count4"}).set_index("spirit", drop=False)
ing4_cnt.head()

# strIngredient5
spirits_ing5 = drinks_df[drinks_df["strIngredient5"].isin(spirit_list)]
ing_cnt_df_5=spirits_ing5.groupby('strIngredient5').nunique()
ing5_cnt = ing_cnt_df_5[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient5":"spirit","idDrink":"count5"}).set_index("spirit", drop=False)
ing5_cnt.head()

# strIngredient6
spirits_ing6 = drinks_df[drinks_df["strIngredient6"].isin(spirit_list)]
ing_cnt_df_6=spirits_ing6.groupby('strIngredient6').nunique()
ing6_cnt = ing_cnt_df_6[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient6":"spirit","idDrink":"count6"}).set_index("spirit", drop=False)
ing6_cnt.head()

# strIngredient7
spirits_ing7 = drinks_df[drinks_df["strIngredient7"].isin(spirit_list)]
ing_cnt_df_7=spirits_ing7.groupby('strIngredient7').nunique()
ing7_cnt = ing_cnt_df_7[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient7":"spirit","idDrink":"count7"}).set_index("spirit", drop=False)
ing7_cnt.head()

# strIngredient8
spirits_ing8 = drinks_df[drinks_df["strIngredient8"].isin(spirit_list)]
ing_cnt_df_8=spirits_ing8.groupby('strIngredient8').nunique()
ing8_cnt = ing_cnt_df_8[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient8":"spirit","idDrink":"count8"}).set_index("spirit", drop=False)
ing8_cnt.head()

# strIngredient9
spirits_ing9 = drinks_df[drinks_df["strIngredient9"].isin(spirit_list)]
ing_cnt_df_9=spirits_ing9.groupby('strIngredient9').nunique()
ing9_cnt = ing_cnt_df_9[["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient9":"spirit","idDrink":"count9"}).set_index("spirit", drop=False)
ing9_cnt.head()

# strIngredient10
spirits_ing10 = drinks_df[drinks_df["strIngredient10"].isin(spirit_list)]
ing_cnt_df_10 =spirits_ing10.groupby('strIngredient10').nunique()
ing10_cnt = ing_cnt_df_10 [["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient10":"spirit","idDrink":"count10"}).set_index("spirit", drop=False)
ing10_cnt.head()

# strIngredient11
spirits_ing11 = drinks_df[drinks_df["strIngredient11"].isin(spirit_list)]
ing_cnt_df_11 =spirits_ing11.groupby('strIngredient11').nunique()
ing11_cnt = ing_cnt_df_11 [["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient11":"spirit","idDrink":"count11"}).set_index("spirit", drop=False)
ing11_cnt.head()

# strIngredient12
spirits_ing12 = drinks_df[drinks_df["strIngredient12"].isin(spirit_list)]
ing_cnt_df_12 =spirits_ing12.groupby('strIngredient12').nunique()
ing12_cnt = ing_cnt_df_12 [["idDrink"]].reset_index(drop=False).rename(columns={"strIngredient12":"spirit","idDrink":"count12"}).set_index("spirit", drop=False)
ing12_cnt.head()



# Concat count columns
# print (pd.concat([ing1_cnt, ing2_cnt, ing3_cnt, ing4_cnt, ing5_cnt, ing6_cnt, ing7_cnt, ing8_cnt, ing9_cnt, ing10_cnt, ing11_cnt, ing12_cnt], ignore_index=False))
spirit_count_df = pd.concat([ing1_cnt, ing2_cnt, ing3_cnt, ing4_cnt, ing5_cnt, ing6_cnt, ing7_cnt, ing8_cnt, ing9_cnt, ing10_cnt, ing11_cnt, ing12_cnt], ignore_index=False)
spirit_count_df.head()


,spirit,count1,count2,count3,count4,count5,count6,count7,count8,count9,count10,count11,count12
spirit,,,,,,,,,,,,,
Advocaat,Advocaat,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Amaretto,Amaretto,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Blue Curacao,Blue Curacao,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bourbon,Bourbon,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brandy,Brandy,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# Add a column for total count
spirit_count_df ['total'] = spirit_count_df.sum(axis=1)
spirit_count_df.head(50)

,spirit,count1,count2,count3,count4,count5,count6,count7,count8,count9,count10,count11,count12,total
spirit,,,,,,,,,,,,,,
Advocaat,Advocaat,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Amaretto,Amaretto,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0
Blue Curacao,Blue Curacao,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Bourbon,Bourbon,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
Brandy,Brandy,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0
Champagne,Champagne,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
Cherry Heering,Cherry Heering,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Cognac,Cognac,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
Cointreau,Cointreau,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


In [28]:
# Drop all columns except Total
cols_to_keep = ["spirit","total"]
spirit_total_df = spirit_count_df[cols_to_keep]
spirit_total_df.head(50)

,spirit,total
spirit,,
Advocaat,Advocaat,1.0
Amaretto,Amaretto,19.0
Blue Curacao,Blue Curacao,1.0
Bourbon,Bourbon,7.0
Brandy,Brandy,13.0
Champagne,Champagne,4.0
Cherry Heering,Cherry Heering,1.0
Cognac,Cognac,2.0
Cointreau,Cointreau,2.0


In [29]:
# Use above df with total and save to csv (for bubble chart) https://www.d3-graph-gallery.com/graph/circularpacking_template.html
# spirit_total_df.to_csv("spirit_total.csv", index=False, header=True)

In [30]:
spirit_total_df.to_sql(name='spirit_totals', con=engine, if_exists='replace', index=False)